## REQUIRED PACKAGGES

In [ ]:
# Machine learning packages
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape, Activation, LSTM, TimeDistributed, GlobalMaxPool2D, BatchNormalization, Dropout, InputLayer
from keras.callbacks import EarlyStopping
from keras import backend as k

# Dataset management packages
from spivutils.synthetic_datasets.spid import load_data
from spivutils.batch_generators.keras_generator import batch_data
from spivutils.common_tools.operations import normalization, vectoraddition, thresholding, imagecropping

# General purpose packages
import numpy as np
import gc

## HARDWARE SETTINGS

In [ ]:
# Detects the avaiable hardware
cpu_devices = tf.config.list_physical_devices('CPU')
gpu_devices = tf.config.experimental.list_physical_devices('GPU')

# Allow memory growth
for gpu in gpu_devices:
  tf.config.experimental.set_memory_growth(gpu, True)

## MODEL HYPERPARAMETERS

In [ ]:
# CNN Layers Hyperparameters
cnn_layers = 3
filters = [32, 64, 128, 256, 512, 1024, 2048]
kernel_size = [(7,7), (5,5), (3,3), (3,3), (3,3)]
pool_size = [(6,6), (4,4), (2,2), (2,2), (2,2)]
padding = 'same'
momentum = 0.9

# Dense Layers Hyperparameters
dense_layers = 7                                        # Number of Dense Layers
dense_units = [2048, 1024, 512, 256, 128, 64, 32]     # Hidden units per layer

# LSTM Layers Hyperparameters
lstm_layers = 3                                         # Number of LSTM Layers
lstm_units = [32, 64, 128, 256, 512, 1024]              # Hidden units per layer

# Training Hyperparameters
chunk_size = 100 #15119                                        # Slice of the dataset
batch_size = 5                                      # Number of patterns shown to the network before weight matrix update
epochs = 10                                             # Number of times that the model sees the dataset
activation = 'LeakyReLU', 'linear'                           # Sets the activation functions
optimizer_function = 'adam'                                      # Weight update after each iteration
loss_function = 'mean_squared_error'                             # Sets the Loss function
learning_rate = 0.001                                           
dropout = 0.5

## DATA MANAGING

In [ ]:
# Data importing
(train_x, train_y), (valid_x, valid_y), _ = load_data()

# Collect a chunk of the dataset
train_x_chunk = train_x[0:int(chunk_size*0.7)]
train_y_chunk = train_y[0:int(chunk_size*0.7)]
valid_x_chunk = valid_x[0:int(chunk_size*0.15)]
valid_y_chunk = valid_y[0:int(chunk_size*0.15)]

# Load data in batches to avoid memory overload
train_batch = batch_data(train_x_chunk, train_y_chunk, batch_size)
valid_batch = batch_data(valid_x_chunk, valid_y_chunk, batch_size)

## Pre-processing

In [ ]:
def standardization(input_data):

    output_data = np.zeros(input_data.shape)

    #min = np.min(input_data)
    #max = np.max(input_data)

    max = 8.75
    min = -8.75

    for i in range(2):

        output_data[:, i, :, :] = (input_data[:, i, :, :] - min)/(max - min)

    return output_data

In [ ]:
train_batch.add_x_preprocessing_operation(imagecropping)
valid_batch.add_x_preprocessing_operation(imagecropping)

train_batch.add_y_preprocessing_operation(imagecropping)
valid_batch.add_y_preprocessing_operation(imagecropping)

train_batch.add_x_preprocessing_operation(normalization)
valid_batch.add_x_preprocessing_operation(normalization)

train_batch.add_y_preprocessing_operation(standardization)
valid_batch.add_y_preprocessing_operation(standardization)

train_batch.add_y_preprocessing_operation(vectoraddition)
valid_batch.add_y_preprocessing_operation(vectoraddition)

## Model Definition

In [ ]:
# Convolutional neural network setup
def build_cnn_layers(input_shape, filters, kernel_size, pool_size, activation, padding, momentum, num_layers):

    model = Sequential()
    model.add(InputLayer(input_shape = input_shape))

    for layer in range(0, num_layers):
        model.add(Conv2D(filters = filters[layer], kernel_size = kernel_size[layer], padding = padding))
        model.add(Activation(activation[0]))
        model.add(BatchNormalization(momentum = momentum))
        model.add(MaxPooling2D(pool_size = pool_size[layer]))
       
    model.add(Flatten())

    return model

In [ ]:
def build_lstm_layers(model, lstm_units, num_layers):

    for layer in range(0, num_layers - 1):

        model.add(LSTM(units = lstm_units[layer], return_sequences = True))

    model.add(LSTM(units = lstm_units[layer], return_sequences = False))

In [ ]:
def build_dense_layers(model, dense_units, activation, num_layers):

    for layer in range(0, num_layers):

        model.add(Dense(units = dense_units[layer]))
        model.add(Activation(activation[0]))  

In [ ]:
def build_model(input_shape, output_shape, filters, kernel_size, pool_size, lstm_units, dense_units, activation, padding):

    convnet = build_cnn_layers(input_shape[1:], filters, kernel_size, pool_size, activation, padding, momentum, cnn_layers)

    model = Sequential()

    model.add(TimeDistributed(convnet, input_shape = input_shape))

    build_lstm_layers(model, lstm_units, lstm_layers)

    build_dense_layers(model, dense_units, activation, dense_layers)

    model.add(Dense(np.prod(output_shape)))
    model.add(Activation(activation[1]))

    model.add(Reshape(output_shape))

    return model

In [ ]:
# Model dimensions
input_shape = train_batch[0][0][0,].shape
output_shape = train_batch[0][1][0,].shape

model = build_model(input_shape, output_shape, filters, kernel_size, pool_size, lstm_units, dense_units, activation, padding)

model.compile(loss = loss_function, optimizer = optimizer_function)

## MODEL SUMMARIZATION

In [ ]:
model.summary()

## MODEL TRAINING

In [ ]:
# Defines an Early Stop callback
es_callback = EarlyStopping(monitor = 'val_loss', patience = 3)

class ClearMemory(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        gc.collect()
        k.clear_session()

In [ ]:
model.fit(train_batch, validation_data = valid_batch, epochs = epochs, callbacks = [ClearMemory(), es_callback])

In [ ]:
#model.save('model_2.keras')